<a href="https://colab.research.google.com/github/sahug/pyspark/blob/main/Spark%20-%20PySpark%20-%20Simple%20ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spark - PySpark - Simple ML**

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 50.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d87f1af27db9de6ec16d66c9c79b9e4744c1dcb0bb45b2c1a171e877da239962
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SimpleML").getOrCreate()

In [6]:
df_pyspark = spark.read.csv("/content/salary.csv", header=True, inferSchema=True)
df_pyspark.show()

+---------+---+---+------+
|     Name|Age|Exp|Salary|
+---------+---+---+------+
|    Krish| 31| 10| 30000|
|Sudhanshu| 30|  8| 25000|
|    Sunny| 29|  4| 20000|
|     Paul| 24|  3| 20000|
|   Harsha| 21|  1| 15000|
|  Shubham| 23|  2| 18000|
+---------+---+---+------+



In [7]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Exp: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [17]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(
    inputCols=["Age", "Exp"],
    outputCol="Ind Features"
    )

output = featureassembler.transform(df_pyspark)

output.show()

output.columns

+---------+---+---+------+------------+
|     Name|Age|Exp|Salary|Ind Features|
+---------+---+---+------+------------+
|    Krish| 31| 10| 30000| [31.0,10.0]|
|Sudhanshu| 30|  8| 25000|  [30.0,8.0]|
|    Sunny| 29|  4| 20000|  [29.0,4.0]|
|     Paul| 24|  3| 20000|  [24.0,3.0]|
|   Harsha| 21|  1| 15000|  [21.0,1.0]|
|  Shubham| 23|  2| 18000|  [23.0,2.0]|
+---------+---+---+------+------------+



['Name', 'Age', 'Exp', 'Salary', 'Ind Features']

In [18]:
final_ds = output.select("Ind Features", "Salary")

final_ds.show()

+------------+------+
|Ind Features|Salary|
+------------+------+
| [31.0,10.0]| 30000|
|  [30.0,8.0]| 25000|
|  [29.0,4.0]| 20000|
|  [24.0,3.0]| 20000|
|  [21.0,1.0]| 15000|
|  [23.0,2.0]| 18000|
+------------+------+



In [20]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = final_ds.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Ind Features", labelCol="Salary")
regressor = regressor.fit(train_data)

In [21]:
regressor.coefficients

DenseVector([-115.8558, 1602.7301])

In [22]:
regressor.intercept

16904.795239761996

In [ ]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

In [ ]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError